In [1]:
import (
	"encoding/csv"
	"fmt"
	"log"
	"os"
	"sort"
	"strconv"
)

// Define a structure to hold location data
type Location struct {
	Name             string
	AverageTemperature      float64
	NrOfPredators  float64
	NrFoodSources float64
	NrWaterSources     float64
	NrHumans    float64
	VegetationType       float64
	LandSize         float64
}

In [2]:
// Function to calculate similarity score
func calculateSimilarity(ideal Location, other Location) float64 {
	// Define the ranges of the features
	ranges := map[string]float64{
		"AverageTemperature":  40.0,
		"NrOfPredators":       20.0,
		"NrFoodSources":       20.0,
		"NrWaterSources":      10.0,
		"NrHumans":            50.0,
		"VegetationType":      6.0,
		"LandSize":            1000.0,
	}

	// Normalize the features
	idealVector := []float64{
		ideal.AverageTemperature / ranges["AverageTemperature"],
		ideal.NrOfPredators / ranges["NrOfPredators"],
		ideal.NrFoodSources / ranges["NrFoodSources"],
		ideal.NrWaterSources / ranges["NrWaterSources"],
		ideal.NrHumans / ranges["NrHumans"],
		ideal.VegetationType / ranges["VegetationType"],
		ideal.LandSize / ranges["LandSize"],
	}
	otherVector := []float64{
		other.AverageTemperature / ranges["AverageTemperature"],
		other.NrOfPredators / ranges["NrOfPredators"],
		other.NrFoodSources / ranges["NrFoodSources"],
		other.NrWaterSources / ranges["NrWaterSources"],
		other.NrHumans / ranges["NrHumans"],
		other.VegetationType / ranges["VegetationType"],
		other.LandSize / ranges["LandSize"],
	}

	return 1 - floats.Distance(idealVector, otherVector, 2) // Using Euclidean distance
}

In [3]:
// Helper function to encode categorical variables for VegetationType column
func encodeCategory(value string, categories map[string]float64) float64 {
	return categories[value]
}

In [4]:
// Function to recommend locations based on a given location
func recommendBasedOnLocation(locations []Location, givenLocation string) []struct {
	Location   Location
	Similarity float64
} {
	// Find the given location
	var ideal Location
	for _, location := range locations {
		if location.Name == givenLocation {
			ideal = location
			break
		}
	}
	if ideal.Name == "" {
		fmt.Printf("Location '%s' not found.\n", givenLocation)
		return nil
	}

	recommendations := []struct {
		Location   Location
		Similarity float64
	}{}

	for _, location := range locations {
		if location.Name != ideal.Name {
			similarity := calculateSimilarity(ideal, location)
			recommendations = append(recommendations, struct {
				Location   Location
				Similarity float64
			}{location, similarity})
		}
	}

	// Sort recommendations by similarity
	sort.Slice(recommendations, func(i, j int) bool {
		return recommendations[i].Similarity > recommendations[j].Similarity
	})

	return recommendations
}

In [5]:
func main() {
	// Open the CSV file
	file, err := os.Open("gopher_locations.csv")
	if err != nil {
		log.Fatal(err)
	}
	defer file.Close()

	// Create a new CSV reader reading from the opened file
	reader := csv.NewReader(file)
	records, err := reader.ReadAll()
	if err != nil {
		log.Fatal(err)
	}

    vegetationCategories := map[string]float64{"Fields": 0, "Meadow": 1, "Hills": 2, "Forest": 3, "Wetland": 4, "Savanna": 5, "Desert": 6}


	// Parse the records into locations
	locations := []Location{}
	for _, record := range records[1:] { // Skip header
		averageTemp, _ := strconv.ParseFloat(record[1], 64)
        nrOfPredators, _ := strconv.ParseFloat(record[2], 64)
        nrFoodSources, _ := strconv.ParseFloat(record[3], 64)
        nrWaterSources, _ := strconv.ParseFloat(record[4],64)
        nrHumans, _ := strconv.ParseFloat(record[5],64)
        vegetationType := encodeCategory(record[6], vegetationCategories)
		landSize, _ := strconv.ParseFloat(record[7], 64)

		location := Location{
			Name:             record[0],
			AverageTemperature:      averageTemp,
			NrOfPredators:  nrOfPredators,
			NrFoodSources: nrFoodSources,
			NrWaterSources:     nrWaterSources,
			NrHumans:    nrHumans,
			VegetationType:       vegetationType,
			LandSize:         landSize,
		}
		locations = append(locations, location)
	}

	// Get recommendations based on the given location
	recommendations := recommendBasedOnLocation(locations, "Stikker Creek")

	// Print the top recommendations
	if recommendations != nil {
		fmt.Println("Top recommended locations:")
		for _, rec := range recommendations[:5] {
			fmt.Printf("%s (Similarity: %.2f)\n", rec.Location.Name, rec.Similarity)
		}
	}
}

Top recommended locations:
Ideal Location (Similarity: 0.86)
Steep Highlands (Similarity: 0.68)
Serene Clearing (Similarity: 0.63)
Green Highlands (Similarity: 0.62)
Sunny Grassland (Similarity: 0.61)
